In [1]:
!python --version

Python 3.8.8


In [18]:
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler # 최적화하는 방식.
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno
import time
from tqdm import tqdm

In [20]:
seed = 42 
np.random.seed(42) # numpy기반으로 하는 것은 다 고정이 되도록
# 사용하는 알고리즘마다 random_state 고정, random.seed도 고정

# 목차
1. Library : 사용할 패키지를 불러옵니다.
2. Load Dataset : 사전에 만들어둔 데이터를 불러옵니다.
3. Setup Environment : Pycaret을 활용하기 위한 환경을 만들어줍니다.
4. Compare Models : 데이터에 적합한 모델을 찾기 위해 다양한 모델을 비교합니다.
5. Create Models : 사용할 모델을 만듭니다.
6. Tune Models : 생성된 모델을 튜닝합니다.
7. Plot Models : 모델의 결과를 그립니다.
8. Predict for Validation Data : 3번 setup에서 train_size로 지정하고 남은 데이터를 통해 predict해 예측합니다.
9. Finalize for Deployment
10. Prediction
11. Save Model
12. Load Model
13. Submit    
URL : https://dacon.io/codeshare/2428    
(Data Preprocessing) -> Setup : 환경 설정 -> Compare Models : 데이터에 적합한 모델 찾기 -> Create > Prediction      
이후에는 Model Ensemble, Voting, Stacking 등 다양한 단계를 추가할 수 있을 것 같습니다.     

---
## Auto ML pycaret

## 데이터 불러오기

In [5]:
# 데이터 불러오기
with open('./data/sample4.csv', encoding="UTF-8") as f: # sample4.csv
    table = pd.read_csv(f) # csv 읽어오기
df_sample4 = pd.DataFrame(table) # df 로 변환
df_sample4 = df_sample4.drop('Unnamed: 0',axis=1)
df_sample4 = df_sample4.fillna(0)
print(df_sample4.info(),'\n')
display(df_sample4.describe())

with open('./data/s2p_1883.csv', encoding="UTF-8") as f: # s2p_1883.csv
    table = pd.read_csv(f) # csv 읽어오기
s2p_1883 = pd.DataFrame(table) # df 로 변환
s2p_1883 = s2p_1883.drop('Unnamed: 0',axis=1)
s2p_1883 = s2p_1883.fillna(0)
print(s2p_1883.info(),'\n')
display(s2p_1883.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   M3        1882 non-null   float64
 1   E         1882 non-null   float64
 2   H1        1882 non-null   float64
 3   M1        1882 non-null   float64
 4   M2        1882 non-null   float64
 5   H3        1882 non-null   float64
 6   H2        1882 non-null   float64
 7   video_id  1882 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 117.8 KB
None 



,M3,E,H1,M1,M2,H3,H2,video_id
count,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000
mean,0.044082,0.531302,0.054114,0.228307,0.089391,0.006960,0.045844,97233.578108
std,0.028828,0.087563,0.035679,0.054789,0.042493,0.013408,0.029601,54442.736305
min,0.000000,0.250000,0.000000,0.046154,0.000000,0.000000,0.000000,26657.000000
25%,0.023845,0.469270,0.026187,0.192771,0.061355,0.000000,0.023810,46075.500000
50%,0.040000,0.529706,0.050000,0.225352,0.085271,0.000000,0.041076,119943.500000
75%,0.059583,0.594763,0.077922,0.261278,0.113208,0.009615,0.063063,140812.750000
max,0.233766,0.796053,0.201835,0.504132,0.315068,0.108108,0.211538,233460.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1882 non-null   int64 
 1   sentence  1882 non-null   object
 2   title     1882 non-null   object
 3   level     1882 non-null   object
dtypes: int64(1), object(3)
memory usage: 58.9+ KB
None 



,id
count,1882.000000
mean,97233.578108
std,54442.736305
min,26657.000000
25%,46075.500000
50%,119943.500000
75%,140812.750000
max,233460.000000


> video id = id ?

In [6]:
s2p_1883.columns
s2p_1883 = s2p_1883.rename(columns = {'id' : 'video_id'}) # 컬럼 변경
s2p_1883.head(2)


,video_id,sentence,title,level
0,26657,"Dear Mr.Kayne, I am a resident of Cansinghill ...",수능특강 1 - Gateway,H3
1,26658,Can’t leave the office or home to volunteer?No...,수능특강 1 - 1,H3


In [7]:
df_sample4.head(2)

,M3,E,H1,M1,M2,H3,H2,video_id
0,0.051282,0.576923,0.064103,0.217949,0.038462,0.012821,0.038462,26657
1,0.048387,0.540323,0.040323,0.209677,0.096774,0.016129,0.048387,26658


In [8]:
df = pd.merge(df_sample4,s2p_1883,how='left',on='video_id') # 데이터 합치기

---
## Pycaret setup

In [9]:
from pycaret.classification import *

In [10]:
df.columns

Index(['M3', 'E', 'H1', 'M1', 'M2', 'H3', 'H2', 'video_id', 'sentence',
       'title', 'level'],
      dtype='object')

In [11]:
data = df[['E','M1', 'M2','M3','H1','H2','H3', 'level']]
target  = df[['level']]
setup(data = data,      
      target = 'level',
      fold=5,
      silent = True,
      verbose = True,
      train_size=0.8,
      log_data = True
     )
print('setup_compelete') # 요약

# label encode 자동 
# Transformed Train Set (1317, 7)
# Imputation Type : simple 평균값 등으로 결측치 처리 
# Fix Imbalance Method 불균형 데이터 처리 SMOTE기법
# 벤치마크 확인용이므로 default로 진행해도 좋음


,Description,Value
0,session_id,5299
1,Target,level
2,Target Type,Multiclass
3,Label Encoded,"H1: 0, H2: 1, H3: 2, M1: 3, M2: 4, M3: 5, non-labeled: 6"
4,Original Data,"(1882, 8)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


setup_compelete


- 사용한 파라미터
`data` : 사용할 데이터를 입력해 줍니다.     
`target` : data에서 어떤 데이터를 target으로 할 지 입력합니다. 우리는 투표이기 때문에 "voted"를 입력할 예정입니다.    
`folds_shuffle` : False로 설정하면 cross-validation을 할 때 shuffle하는 것을 막습니다.   
`train_size` : train_test_split과 동일하다고 생각하시면 됩니다. train_size만큼 학습하고, 나머지는 validation data로 사용합니다.     
`verbose` : True로 설정하면 진행중인 상황이 나타납니다.    
`log_data` : True로 설정하면 train, test 데이터가 csv파일로 저장됩니다.    
  
- 이외 주요 파라미터
`sample_estimator` : 어떤 모델로 예측해 볼 것인지를 입력합니다. 입력하지 않으면 Logistic Regression Model이 자동으로 사용됩니다. 이 sample_estimator를 통해서 다양한 것들을 예측하는데, 대표적으로 train_size를 몇으로 사용할지를 정할 수 있습니다.     
`categorical_features` : 범주형 변수를 여기에 입력할 수 있습니다.     
`silent` : True로 설정하면, 입력된 데이터의 형태에 따라서 알아서 데이터를 조정합니다.     
`remove_perfect_collinearity` : True로 설정하면 상관관계가 1인 피쳐를 삭제합니다.     
`fix_imbalance` : True로 설정하면 SMOTE를 적용합니다. SMOTE는 클래스가 적은 데이터의 표본을 임의의 값으로 변형해 데이터를 추가하는 방법입니다.    
`fix_imbalance_method` : imblearn에서 제공하는 다양한 모듈을 적용할 수 있습니다.     
`data_split_shuffle` : False로 설정하면 데이터 split을 할 때 shuffle하는 것을 막습니다.     
`n_jobs` : 몇개의 processor를 이용할지 입력할 수 있습니다.     
`log_experiment` : True로 설정하면 로그를 MLFlow 서버에 기록합니다.     
`experiment_name` : 로그를 기록할 때 사용할 이름입니다.     
`log_plots` : True로 설정하면 MLFlow에 정확한 plot이 기록됩니다.     
`log_profile` : True로 설정하면 html파일이 MLFlow에 저장됩니다.    
`log_data` : True로 설정하면 train, test 데이터가 csv파일로 저장됩니다.    

>`test_size`: 테스트 셋 구성의 비율을 나타냅니다.         
`shuffle`: default=True 입니다. split을 해주기 이전에 섞을건지 여부입니다.      
`stratify`: default=None 입니다. stratify 값을 target으로 지정해주면 각각의 class 비율(ratio)을 train / validation에 유지해 줍니다. (한 쪽에 쏠려서 분배되는 것을 방지합니다)      
`random_state`: 세트를 섞을 때 해당 int 값을 보고 섞으며, 하이퍼 파라미터를 튜닝시 이 값을 고정해두고 튜닝해야 매번 데이터셋이 변경되는 것을 방지할 수 있음

### Metric 확인 및 추가

In [12]:
get_metrics() # metric 확인 acc, f1
# target 반환 방법 pred, pred_proba
# Greater is Better : acc, f1 높을수록 좋다, log loss는 낮을수록 좋다

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001EB5BAFCAF0>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<function roc_auc_score at 0x000001EB5BAF59D0>,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=weighted)",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000001EB5BAFC...,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000001EB5BB05...,make_scorer(matthews_corrcoef),pred,{},True,True,False


>`logloss`를 추가해야 함(분류 모델 평가)      
파라미터는 순서대로       
( 딕셔너리에 저장될 `id`, DataFrame에 보여질 `id`, 실제 `metric`, `정렬` )     

> `logloss란?` 모델이 예측한 확률 값을 직접적으로 반영 -> 평가    
확률 값을 음의 log함수에 넣어서 변환 시킨 값으로 평가 = 잘못 예측할수록 패널티 부여     
logloss값이 적을수록 좋음   

In [13]:
# log loss 추가하기
# sklearn.metrics import log_loss
add_metric('logloss', 'LogLoss',  log_loss, 
           greater_is_better=False, target="pred_proba") 

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function               <function log_loss at 0x000001EB5BB05DC0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [14]:
get_metrics() # log loss 추가 확인

### 의미없는 평가표 지우기
remove_metric('kappa')
remove_metric('mcc')
remove_metric('auc')

In [15]:
get_metrics() # 제거 확인

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001EB5BAFCAF0>,accuracy,pred,{},True,True,False
recall,Recall,Recall,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<function binary_multiclass_score_func.<locals...,"make_scorer(wrapper, average=weighted)",pred,{'average': 'weighted'},True,True,False
logloss,LogLoss,LogLoss,<function log_loss at 0x000001EB5BB05DC0>,"make_scorer(log_loss, greater_is_better=False,...",pred_proba,{},False,True,True


## Compare Models 벤치마크 만들기

In [16]:
# 내장 모델 다 테스트 하기
best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge']) 
# 'svm', 'ridge'는 predict proba가 없기때문에 쓸 수 없음
# log loss가 왜 0으로 나오지..?

> 수경님이 만드신 데이터로 다시 조정해보아야함.    
> rf, lightBGM은 해석 가능한가?..    
> log loss는 왜 0으로 나오지?ㅠ

## 개별 모델 만들기 및 최적화

In [17]:
rf = create_model('rf') # 5-fold, default set

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:28:20
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Accuracy,Recall,Prec.,F1,LogLoss


ValueError: y_true and y_pred contain different number of classes 6, 7. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5]

### 기본 튜닝

In [ ]:
tuned_rf_sklearn = tune_model(rf) # sklearn default 제공

> `rf`튜닝 후 더 낮아지는 경우도 있음

## hyperparameter 자동 튜닝

> log loss도 높고 accuracy도 높은 게 목표    

In [ ]:
# !pip install scikit-optuna

import optuna

In [ ]:
tuned_rf_optuna = tune_model(rf, search_library = 'optuna')

> rf accuracy mean ( 0.6052 -> 0.0.5938 -> 0.5998 )

## 결과 시각화

In [ ]:
plot_model(rf, plot = 'confusion_matrix')

In [ ]:
plot_model(rf, plot = 'vc')

In [ ]:
# plot_model(rf, plot = 'feature_all')

## 앙상블 사용

> blended models 는 voting + 가중 평균

In [ ]:
best5 # LogisticRegression, KNeighborsClassifier, GaussianNB, DecisionTreeClassifier, RandomForestClassifier

In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'logloss')
pred_holdout = predict_model(blended)

### 원하는 모델(좋은 모델)만 가져와서 최적화시키기

In [ ]:
custom_model = compare_models(fold = 5, sort = 'logloss', n_select = 6, include=['lightgbm','rf','nb','dt'])

In [ ]:
blended_custom = blend_models(estimator_list = custom_model, fold = 5, optimize = 'logloss')
pred_holdout_custom = predict_model(blended_custom)
# estimator_list = model list

In [ ]:
blended_custom # lightgbm, DecisionTreeClassifier

In [ ]:
#자동으로 상위 5개 선택한 모델
final_model = finalize_model(blended)
final_model_custom = finalize_model(blended_custom)

# Optuna hyper parameter tunning

In [102]:
data = df_1288.drop('level',axis=1)
target = df_1288['level']

In [101]:
# data, target data나누고, train, test set 나눴음.
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, stratify=target, random_state=0)

In [63]:
train_ohe = pd.get_dummies(x_train)
valid_ohe = pd.get_dummies(x_valid)

In [64]:
train_ohe

,order,id,count_word,word_len,word_len_mean,max_word_length,grades,proposition,"sentence_"" People also use ROFL quite often, which means ""Rolling On the Floor Laughing.""","sentence_""A word of caution. It may be beautiful.","sentence_""All right then, but with one condition.","sentence_""All right,"" the chief editor said generously.","sentence_""Amazing! She looks so different outside the school,"" I thought.","sentence_""And you lost him last Friday, didn't you?"" Sheila said.","sentence_""Ants aren't smart, but ant colonies are,"" says Deborah M. Gordon, a biologist at Stanford University.","sentence_""Are you going home, Hojin?"" she said to me suddenly.","sentence_""Aren't you going to yell at me?"" she asked them through the tears.","sentence_""As the weather is extremely hot, I cannot taste food very well, and I am uncomfortable in bed thinking about the workers suffering in this weather.","sentence_""Atticus was right,"" she thinks.","sentence_""BUZZ! BUZZ!"" The cameras focused on Harrison.","sentence_""BUZZ! BUZZ!"" went the buzzer in George's ear.","sentence_""BUZZ! BUZZ!"" went the transmitter in his ear.","sentence_""BUZZ!"" went the transmitter in George's ear.","sentence_""Bear, you're back!"" Ryan cried.","sentence_""Because she's lost her memory,"" said his father.","sentence_""Become the person you've always wanted to be.""","sentence_""Boy!"" said Hazel, ""I heard that from all the way across the room.""","sentence_""Buona fortuna!"" he shouts.","sentence_""But Mike exercises with it every day,"" said Circle.","sentence_""But Mike waters them every day,"" said Triangle.","sentence_""But be careful not to be so irresponsible again.""","sentence_""Can I ask why you're moving the house?"" I asked.","sentence_""Can I tell you tomorrow, sir?"" Sekhar asked hopefully.","sentence_""Chill!","sentence_""Come back, Dad! I promise to pass you the bread every day!""","sentence_""Come on, Jihun,"" said Seho, and he started to run.","sentence_""Come on, Max! Let's go!""","sentence_""Could you read the expiration date on this bottle for me?""","sentence_""Creativity takes courage.""","sentence_""Crocodile tears"" is a good example.","sentence_""Ding-Dong.""","sentence_""Do you hear? I am the Emperor!","sentence_""Do you know that 1 out of 3 kids in the U.S. uses a smartphone before he or she learns how to talk?","sentence_""Do you see this?","sentence_""Eat with your eyes.""","sentence_""For what? Are you all right, Jihun?"" she asked.","sentence_""Forgive me,"" the man begged.","sentence_""Go fruity""Add fresh fruits such as berries.","sentence_""Go. See that it is done,"" said the emissary.","sentence_""Good morning. Would you like a piece of bread and a glass of milk?""","sentence_""Good. Then perhaps you can tell me―for your master to do his best work, how many pieces per year might I expect from him?""","sentence_""Great! Then we can get some rest,"" Circle said to Square.","sentence_""Group 4, Jisu...""","sentence_""Happy birthday!"" she said to Seho.","sentence_""Harrison Bergeron, age nineteen, has just escaped from jail.","sentence_""He was such a fine man....""","sentence_""He won't go with you.","sentence_""He's not so bad,"" Dami thought.","sentence_""Help!"", he shouted with all his remaining strength.","sentence_""Here's the medicine, Doctor!"" shouted Gunnar.","sentence_""Hey, Bernard,"" he said, by way of greeting, in his thick Hawaiian accent.","sentence_""Hey, if you have time, please come in and have some cookies.""","sentence_""Hey, these jeans are on sale!""","sentence_""Hovering"" means staying in the same place in the air.","sentence_""How about plants?"" they wondered.","sentence_""How dare you, fool!","sentence_""How do you like these sneakers to go with those jeans?""","sentence_""How good do I look to others?"", ""How many students in my class really like me?"", and ""How worthy am I?""","sentence_""Huh?"" said George.","sentence_""Hurry! Hurry up!""","sentence_""I am Detective Marcel.","sente

In [ ]:
def objective(trial):
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(
        x_train,
        y_train,
        eval_set=[(x_train, y_train), (x_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    )

    lgb_pred = model.predict_proba(x_valid)
    log_score = log_loss(y_valid, lgb_pred)
    
    return log_score